In [2]:
!python -V

Python 3.10.4


In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [10]:
len(df_train), len(df_val)

(73908, 61921)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.7587152055230355

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [47]:
with mlflow.start_run():

    mlflow.set_tag("developer", "adam")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [15]:
import xgboost as xgb

In [16]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:
import gc
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        del booster
        gc.collect()

    return {'loss': rmse, 'status': STATUS_OK}

In [18]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=20,
    trials=Trials()
)

[14:12:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:13.63730                          
[1]	validation-rmse:9.85054                           
[2]	validation-rmse:8.10029                           
[3]	validation-rmse:7.34188                           
[4]	validation-rmse:7.00284                           
[5]	validation-rmse:6.84461                           
[6]	validation-rmse:6.76610                           
[7]	validation-rmse:6.72636                           
[8]	validation-rmse:6.70055                           
[9]	validation-rmse:6.68745                           
[10]	validation-rmse:6.67882                          
[11]	validation-rmse:6.67289                          
[12]	validation-rmse:6.66760                          
[13]	validation-rmse:6.66518                          
[14]	validation-rmse:6.66291                         

In [20]:
mlflow.xgboost.autolog()
# with mlflow.start_run():
    
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

best_params = {
    'learning_rate': 0.09585355369315604,
    'max_depth': 30,
    'min_child_weight': 1.060597050922164,
    'objective': 'reg:linear',
    'reg_alpha': 0.018060244040060163,
    'reg_lambda': 0.011658731377413597,
    'seed': 42
}

mlflow.log_params(best_params)

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)
mlflow.log_metric("rmse", rmse)

with open("models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)
mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[14:39:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	validation-rmse:6.86273
[28]	valida

2022/05/31 14:40:47 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\adamgifford_behavr.DESKTOP-PQU0D8M\anaconda3\envs\mlopszoom\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils."
2022/05/31 14:40:47 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID ab57ecf614de41bea447e0f8f6339d81. Failed operations: [MlflowException("Changing param values is not allowed. Params were already logged=\'[{\'key\': \'num_boost_round\', \'old_value\': None, \'new_value\': \'1000\'}, {\'key\': \'early_stopping_rounds\', \'old_value\': None, \'new_value\': \'50\'}, {\'key\': \'maximize\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'verbose_eval\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\'

ModelInfo(artifact_path='models_mlflow', flavors={'python_function': {'loader_module': 'mlflow.xgboost', 'python_version': '3.10.4', 'data': 'model.xgb', 'env': 'conda.yaml'}, 'xgboost': {'xgb_version': '1.6.1', 'data': 'model.xgb', 'model_class': 'xgboost.core.Booster', 'code': None}}, model_uri='runs:/ab57ecf614de41bea447e0f8f6339d81/models_mlflow', model_uuid='39fb3e43aa904ef3b7353a7b7fd81456', run_id='ab57ecf614de41bea447e0f8f6339d81', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-05-31 18:40:48.565736', mlflow_version='1.26.0')

In [18]:
mlflow.xgboost.autolog(disable=True)

In [21]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[11:40:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	valid

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [24]:
# mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")
mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle/lin_reg.bin")

PermissionError: [Errno 13] Permission denied: '.\\mlruns\\2\\ab57ecf614de41bea447e0f8f6339d81\\artifacts\\models_pickle\\lin_reg.bin'

In [27]:
from mlflow.tracking import MlflowClient

In [28]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [30]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-week2', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [29]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1654026951601, description='', last_updated_timestamp=1654026987277, latest_versions=[<ModelVersion: creation_timestamp=1654026951630, current_stage='Staging', description='', last_updated_timestamp=1654026987277, name='nyc-taxi-regressor', run_id='ab57ecf614de41bea447e0f8f6339d81', run_link='', source='./mlruns/2/ab57ecf614de41bea447e0f8f6339d81/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>], name='nyc-taxi-regressor', tags={}>]

In [31]:
client.create_experiment(name="my-cool-experiment")

'3'

In [32]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=2,
    filter_string="",  # "metrics.rmse < 6.8"
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"],
)

In [35]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 2e16f5c951394f49ac61e9d911f0894f, rmse: 6.2886
run id: ab57ecf614de41bea447e0f8f6339d81, rmse: 6.3040
run id: eefe47d6d60b4bcfa6ac4fcea8c27b29, rmse: 6.3148
run id: 012e3d4001fc44b58547d7cba4880592, rmse: 6.3218
run id: 2472b5fb14a24e489e630e44489a9a65, rmse: 6.3244


In [36]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# this would work, but not running because I only have one model actually saved as an
# artifact from my experiments, and i've already registered it via the ui
run_id = "2e16f5c951394f49ac61e9d911f0894f"
model_uri = f"runs:/{run_id}/model"
# mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

In [37]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1654026951601, description='', last_updated_timestamp=1654026987277, latest_versions=[<ModelVersion: creation_timestamp=1654026951630, current_stage='Staging', description='', last_updated_timestamp=1654026987277, name='nyc-taxi-regressor', run_id='ab57ecf614de41bea447e0f8f6339d81', run_link='', source='./mlruns/2/ab57ecf614de41bea447e0f8f6339d81/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>], name='nyc-taxi-regressor', tags={}>]

In [39]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging


In [40]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production"
)

<ModelVersion: creation_timestamp=1654026951630, current_stage='Production', description='', last_updated_timestamp=1654027832427, name='nyc-taxi-regressor', run_id='ab57ecf614de41bea447e0f8f6339d81', run_link='', source='./mlruns/2/ab57ecf614de41bea447e0f8f6339d81/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [41]:
from datetime import datetime
model_version = 1
new_stage = "Production"
date = datetime.today()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1654026951630, current_stage='Production', description='model version 1 was transitioned to Production on 2022-05-31 16:13:09.696844', last_updated_timestamp=1654027989697, name='nyc-taxi-regressor', run_id='ab57ecf614de41bea447e0f8f6339d81', run_link='', source='./mlruns/2/ab57ecf614de41bea447e0f8f6339d81/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [ ]:
import pandas as pd

In [42]:
def preprocess(df, dv):
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    train_dicts = df[categorical + numerical].to_dict(orient="records")
    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [43]:
df = read_dataframe("./data/green_tripdata_2021-02.parquet")

In [44]:
run_id = "ab57ecf614de41bea447e0f8f6339d81"
client.download_artifacts(run_id=run_id, path="preprocessor", dst_path=".")

'c:\\Users\\adamgifford_behavr.DESKTOP-PQU0D8M\\Documents\\DataScience\\Tutorials\\mlops-zoomcamp\\02-experiment-tracking\\preprocessor'

In [45]:
with open ("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [46]:
X_test = preprocess(df, dv)

In [47]:
target = "duration"
y_test = df[target].values

In [50]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[10:12:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: total: 18.8 s
Wall time: 1.39 s


{'rmse': 6.303975653496241}

In [ ]:
# copied from video but not run since i have different setup
# this is how you can transition models and archive existing versions in same stage
#  client.transition_model_version_stage(
#     name=model_name,
#     version=4,
#     stage="Production",
#     archive_existing_versions=True
# )